<a href="https://colab.research.google.com/github/chottokun/spaCy_training/blob/main/Spacy_with_electra_learning01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

spaCyでelectraを使ってみる



In [1]:
#in colab cell
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_Mar__8_18:18:20_PST_2022
Cuda compilation tools, release 11.6, V11.6.124
Build cuda_11.6.r11.6/compiler.31057947_0


In [2]:
# GPU
!nvidia-smi

Sun Feb 19 05:55:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    33W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# from transformers import BertJapaneseTokenizer, ElectraForPreTraining
# tokenizer = BertJapaneseTokenizer.from_pretrained('Cinnamon/electra-small-japanese-discriminator', mecab_kwargs={"mecab_option": "-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"})
# model = ElectraForPreTraining.from_pretrained('Cinnamon/electra-small-japanese-discriminator')

In [4]:
# # 
# !pip install -U spacy[cuda112,transformers,lookups,ja]
# !apt-get install mecab mecab-ipadic-utf8
# !pip install mecab-python3 fugashi ipadic
# !pip install unidic-lite

In [5]:
# https://megagonlabs.github.io/ginza/
!pip install -U ginza ja-ginza ja-ginza-electra

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 63.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 KB 16.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 KB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 101.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 93.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 26.3 MB/s eta 0:00:00
  Created wheel for SudachiDict-core: filename=SudachiDict_core-20230110-py3-none-any.whl size=71665

In [6]:
!git clone -b feature/bert_base_japanese_v2 https://github.com/megagonlabs/UD_Japanese-GSD

Cloning into 'UD_Japanese-GSD'...
remote: Enumerating objects: 693, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 693 (delta 7), reused 13 (delta 7), pack-reused 675
Receiving objects: 100% (693/693), 73.31 MiB | 23.13 MiB/s, done.
Resolving deltas: 100% (403/403), done.


In [7]:
!mkdir -p corpus
!python -m spacy convert ./UD_Japanese-GSD/ja_gsd-ud-train.ne.conllu ./corpus
!python -m spacy convert ./UD_Japanese-GSD/ja_gsd-ud-dev.ne.conllu ./corpus
!python -m spacy convert ./UD_Japanese-GSD/ja_gsd-ud-test.ne.conllu ./corpus

!ls ./corpus

2023-02-19 05:56:23.199216: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-19 05:56:24.711183: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-19 05:56:24.711337: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-19 05:56:24.711367: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

In [8]:
import spacy
from spacy.tokens import Doc, DocBin

nlp = spacy.blank("ja") 

train_data = DocBin()
train_data.from_disk(path="./corpus/ja_gsd-ud-train.ne.spacy")

docs = list(train_data.get_docs(nlp.vocab))

for token in docs[2]:
  print("text=%s head=%d dep=%s ent_type=%s ent_job=%s" % (token, token.head.i, token.dep_, token.ent_type_, token.ent_iob_))

text=手 head=2 dep=obl ent_type= ent_job=O
text=に head=0 dep=case ent_type= ent_job=O
text=持っ head=6 dep=acl ent_type= ent_job=O
text=た head=2 dep=aux ent_type= ent_job=O
text=特殊 head=6 dep=acl ent_type= ent_job=O
text=な head=4 dep=aux ent_type= ent_job=O
text=刃物 head=8 dep=obj ent_type= ent_job=O
text=を head=6 dep=case ent_type= ent_job=O
text=使っ head=12 dep=acl ent_type= ent_job=O
text=た head=8 dep=aux ent_type= ent_job=O
text=アクロバティック head=12 dep=acl ent_type= ent_job=O
text=な head=10 dep=aux ent_type= ent_job=O
text=体術 head=22 dep=nmod ent_type= ent_job=O
text=や head=12 dep=case ent_type= ent_job=O
text=、 head=12 dep=punct ent_type= ent_job=O
text=揚羽 head=18 dep=obl ent_type=PERSON ent_job=B
text=と head=15 dep=case ent_type= ent_job=O
text=薄羽 head=18 dep=compound ent_type=PERSON ent_job=B
text=同様 head=24 dep=advcl ent_type= ent_job=O
text=に head=18 dep=aux ent_type= ent_job=O
text=クナイ head=22 dep=nmod ent_type= ent_job=O
text=や head=20 dep=case ent_type= ent_job=O
text=忍具 head=24 de

In [9]:
%%bash
cat << EOF > base_config.cfg
[paths]
train = null
dev = null
vectors = null
[system]
gpu_allocator = "pytorch"

[nlp]
lang = "ja"
pipeline = ["transformer","ner"]
batch_size = 128

[components]

[components.transformer]
factory = "transformer"

[components.transformer.model]
@architectures = "spacy-transformers.TransformerModel.v3"
name = "bert-base-multilingual-uncased"
tokenizer_config = {"use_fast": true}

[components.transformer.model.get_spans]
@span_getters = "spacy-transformers.strided_spans.v1"
window = 128
stride = 96

[components.ner]
factory = "ner"

[components.ner.model]
@architectures = "spacy.TransitionBasedParser.v2"
state_type = "ner"
extra_state_tokens = false
hidden_width = 64
maxout_pieces = 2
use_upper = false
nO = null

[components.ner.model.tok2vec]
@architectures = "spacy-transformers.TransformerListener.v1"
grad_factor = 1.0

[components.ner.model.tok2vec.pooling]
@layers = "reduce_mean.v1"

[corpora]

[corpora.train]
@readers = "spacy.Corpus.v1"
max_length = 0

[corpora.dev]
@readers = "spacy.Corpus.v1"
max_length = 0

[training]
accumulate_gradient = 3
dev_corpus = "corpora.dev"
train_corpus = "corpora.train"

[training.optimizer]
@optimizers = "Adam.v1"

[training.optimizer.learn_rate]
@schedules = "warmup_linear.v1"
warmup_steps = 250
total_steps = 20000
initial_rate = 5e-5

[training.batcher]
@batchers = "spacy.batch_by_padded.v1"
discard_oversize = true
size = 2000
buffer = 256

[initialize]
EOF

In [10]:
# なぜかlocaleがUTF-8じゃないって怒られたので・・・
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [11]:
!python -m spacy init fill-config base_config.cfg config.cfg

2023-02-19 05:57:17.443150: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-19 05:57:17.443261: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-19 05:57:17.443280: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./trai

In [12]:
# corpus/ja_gsd-ud-test.ne.spacy
# ja_gsd-ud-dev.ne.spacy	ja_gsd-ud-test.ne.spacy  ja_gsd-ud-train.ne.spacy

In [13]:
!python -m spacy train config.cfg --gpu-id 0 --output ./ja_ud_japanese_gsd_2.6_ne_trf \
  --paths.train ./corpus/ja_gsd-ud-train.ne.spacy --paths.dev ./corpus/ja_gsd-ud-dev.ne.spacy

2023-02-19 05:57:26.133183: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-19 05:57:26.133304: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-19 05:57:26.133323: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
✔ Created output directory: ja_ud_japanese_gsd_2.6_ne_trf
ℹ Saving to output directory: ja_ud_japanese_gsd_2.6_ne_trf
ℹ Using GPU: 0

=========================== Initi

In [14]:
!ls ja_ud_japanese_gsd_2.6_ne_trf/
# model-best  model-last

model-best  model-last


In [15]:
!mkdir -p ./package
!python -m spacy package ./ja_ud_japanese_gsd_2.6_ne_trf/model-best ./package --name udjagsd26ne_trf --version 0.0.1

2023-02-19 06:36:15.859601: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-19 06:36:15.860219: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-19 06:36:15.860242: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
ℹ Building package artifacts: sdist
✔ Including 1 package requirement(s) from meta and config
spacy-transformers>=1.1.9,<1.2.0
✔ Loaded meta.json from file
ja_ud_japan

In [16]:
from google.colab import files
files.download("./package/ja_udjagsd26ne_trf-0.0.1/dist/ja_udjagsd26ne_trf-0.0.1.tar.gz")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [17]:
!pip install ./package/ja_udjagsd26ne_trf-0.0.1

!pip list | grep udjagsd26ne
# ja-udjagsd26ne-trf            0.0.1 

import spacy
nlp = spacy.load("ja_udjagsd26ne_trf")
nlp.pipeline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./package/ja_udjagsd26ne_trf-0.0.1
  Preparing metadata (setup.py) ... done
  Created wheel for ja-udjagsd26ne-trf: filename=ja_udjagsd26ne_trf-0.0.1-py3-none-any.whl size=622981448 sha256=7415e00c95242f18c6ec09fe9df04e0bcce09de17307501ba9d7fcd0df0b033b
  Stored in directory: /root/.cache/pip/wheels/00/ef/4e/fbc3528a5562dc8ebe84c64910f655f726849a6b005ca0bf5d
Successfully built ja-udjagsd26ne-trf
ja-udjagsd26ne-trf            0.0.1


[('transformer',
  <spacy_transformers.pipeline_component.Transformer at 0x7f2a2009b280>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x7f2a28039e40>)]

In [18]:
%%time
nlp2 = spacy.load("./ja_ud_japanese_gsd_2.6_ne_trf/model-best")

CPU times: user 2.8 s, sys: 1.79 s, total: 4.59 s
Wall time: 4.7 s


In [19]:
%%time
doc = nlp2("""
闇金業者に借金をしてしまうと、元金の返済はおろか、借金が増えていく一方で、心苦しい思いをしてしまうものです。
そして、そのような状態になってしまうと、知りつつ借金したから誰にも頼れない心情となり、一人で悩みを抱え込み、解決策に気づけなくなってしまうことも多々あるのです。
そのような際の解決方法を見つけたいなら、まず他人に相談するのが大切ではないでしょうか。
一人で悩みを抱え込むと、周りが見えなくなってしまい、余計につらくなってしまうことがあります。
そして、そのような時には弁護士や司法書士に相談するのが最良の対処法といえ、無料の相談サービスを利用して悩みを聞いてもらうのがよいのではないでしょうか。
こういった法務事務所・法律事務所ではいつも、ヤミ金問題に困っている人たちの相談に乗っていて、弁護士や司法書士からの助言がもらえるので、とても役立ちます。
ヤミ金解決に数多くの実績がある、人情派弁護士に、あなたの悩み・苦しみを、まずは無料相談してみては？
北海道から沖縄まで、全国対応の事務所だから、東温市に住んでいる方ももちろん相談OKです。
解決に伴う手数料も、分割払いOKなので、今は支払うお金が無くても、安心して相談できます。
ヤミ金に悩んでいる東温市の女性の方、今スグここで無料相談してみませんか？
地元の東温市の近くの法務事務所・法律事務所は下記を参考にどうぞ。
一般の司法書士・弁護士事務所は、通常の債務整理についての相談は受け付けているのですが、闇金の問題に関しては、相談を受け付けていないケースも多くあります。
東温市近辺の弁護士事務所や司法書士事務所でも、そういうところが多いかもしれません。
闇金業者から借金をしてしまった結果、最初こそ普通の督促の電話があったものの後半には脅迫染みた電話がかかってきたり、家に怒鳴り込まれたりするなどあくどい債権回収を行われる場合があります。
このような状況になってしまったら、弁護士や司法書士等に相談することが一番ですが、借金をしていることへの負い目や、闇金業者からの仕返しを恐れて相談や通報をためらってしまう場合もあるようです。
また、弁護士や司法書士に依頼するのに歯止めをかけてしまう理由に、報酬を支払う余裕がない、という場合もあります。
法律関連に強いのが彼らですから、闇金業者が如何に不法かを分かりやすく説明し、現在の目も当てられない状況を解決する手伝いしてくれます。
弁護士や司法書士への依頼費用が高いと悩んでいる方も、無料で相談を受け付けている司法書士事務所や弁護士事務所もあります。
ヤミ金業者からお金を借りてしまうと、元金の返済どころか、借金が増えていくばかりで、ドツボにはまっていってしまうものです。
しかも、闇金にお金を借りるという状況は、ずいぶんお金に追い詰められた状況だけに、そのような人が、ヤミ金業者から借金をして返済できるわけがありません。
そして、そのような状態になってしまうと、知りつつ借りたから人に頼れない心情になり、1人だけで悩みを抱え込んで、打開方法に気づけなくなってしまうこともあります。
そのような時の対策というのは、まず人に相談することが一番です。
""")


CPU times: user 4.86 s, sys: 232 ms, total: 5.09 s
Wall time: 7.51 s


In [20]:
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)


一人 99 101 QUANTITY
一人 181 183 QUANTITY
北海道 431 434 GPE
沖縄 436 438 GPE
東温市 453 456 GPE
東温市 530 533 GPE
東温市 561 564 GPE
東温市 668 671 GPE
弁護士事務所 674 680 FAC
司法書士事務所 681 688 FAC
一番 837 839 ORDINAL
司法書士事務所 1060 1067 FAC
1人 1258 1260 QUANTITY
一番 1323 1325 ORDINAL


In [21]:
doc.ents

(一人, 一人, 北海道, 沖縄, 東温市, 東温市, 東温市, 東温市, 弁護士事務所, 司法書士事務所, 一番, 司法書士事務所, 1人, 一番)

In [22]:
len(doc._.trf_data.tokens["input_texts"])

9

In [23]:
doc._.trf_data.width

768